In [2]:
from __future__ import print_function
import os
import csv
import collections
import string
from time import time
import subprocess
import io
import re
import urllib2
from bs4 import BeautifulSoup
from multiprocessing import Process, Lock, Pool

In [4]:
collection = "clueWeb09_English_1"
indexDir = "/scratch/index/indri_5_7/"
colIndex = os.path.join(indexDir, collection)
projDir = "/home/fj9124/projects/ir/seq_kb_ir/"
configsDir = os.path.join(projDir, "configs")
colConfigsDir = os.path.join(configsDir, collection)
colIndriconfigs = os.path.join(colConfigsDir, "indriRunQuery.cfg")
resultsDir = os.path.join(projDir, "results")
colResultsDir = os.path.join(resultsDir, collection)
qrelsDir = os.path.join(projDir, "qrels")
colQrelsDir = os.path.join(qrelsDir, collection)
precResultsDir = os.path.join(projDir, "precResults")
colPrecResultsDir = os.path.join(resultsDir, collection)
evaluatorsDir = "/home/fj9124/projects/ir/evaluators"


In [11]:
def text2umlsConcepts(origQueries_l):
    #return(type(origQuery))
    origQuery = origQueries_l[1]
    out = subprocess.check_output(["/home/fj9124/thirdPartyProgs/public_mm/bin/testapi.sh", "-AIy+", "-V USAbase", str(origQuery)])
    #print((out))
    f = io.StringIO(unicode(out))
    reader = csv.reader(f, delimiter=':', skipinitialspace=True,)
    #reader1 = reader
    extractedConcepts = [row[1] for row in reader if row[0] in set(['Concept Name', 'Concept Id', 'Semantic Types'])]
    #print(len(extractedConcepts))
    extractedConcepts1 = set([(extractedConcepts[3*i], extractedConcepts[3*i+1].strip().lower(), extractedConcepts[3*i+2].strip(']').strip('[')) for i in range(0, len(extractedConcepts)/3)])
    return([origQueries_l[0], extractedConcepts1])
    

In [12]:
print("getting original concepts")
t0 = time()
origQueries_l = [[k, v] for k, v in origQueries.iteritems()]

pool = Pool(processes=10) 
qidConcepts = (pool.map(text2umlsConcepts, origQueries_l))
print(qidConcepts)
print("done in %0.3f minutes." % ((time() - t0)/60))

getting original concepts
[['24', set([('C0443318', 'sustained', 'ftcn'), ('C0439508', '/year', 'tmco'), ('C0205082', 'severe', 'qlco'), ('C0278140', 'severe pain', 'sosy'), ('C0038000', 'ruptured spleen', 'inpo'), ('C0332174', '/week', 'tmco'), ('C0848377', 'abdominal trauma', 'inpo'), ('C0011923', 'imaging', 'diap'), ('C0857353', 'hypotensive', 'patf'), ('C0687676', 'after', 'tmco'), ('C0701802', 'bike accident', 'inpo'), ('C0150312', 'present', 'qnco'), ('C0000737', 'abdominal pain', 'sosy'), ('C0039231', 'tachycardia', 'fndg'), ('C0443289', 'reveal', 'qlco'), ('C0019065', 'intraperitoneal hemorrhage', 'patf'), ('C0000726', 'abdominal', 'blor'), ('C0205447', 'one', 'qnco'), ('C0580836', 'old', 'tmco'), ('C1706180', 'male', 'qlco')])], ['25', set([('C0019602', 'his', 'aapp, bacs, phsu'), ('C1821572', 'decreased level of consciousness', 'fndg'), ('C0018884', 'helmet', 'mnob'), ('C0087111', 'treatment', 'topp'), ('C0018670', 'head', 'blor'), ('C0205253', 'immediate', 'tmco'), ('C043950

In [13]:
print("convert qidConcepts to a dictionary formatted container")
qidConcepts_d = {qidConcs[0]: list(qidConcs[1]) for qidConcs in qidConcepts} 
print(qidConcepts_d)

convert qidConcepts to a dictionary formatted container
{'24': [('C0443318', 'sustained', 'ftcn'), ('C0439508', '/year', 'tmco'), ('C0205082', 'severe', 'qlco'), ('C0278140', 'severe pain', 'sosy'), ('C0038000', 'ruptured spleen', 'inpo'), ('C0332174', '/week', 'tmco'), ('C0848377', 'abdominal trauma', 'inpo'), ('C0011923', 'imaging', 'diap'), ('C0857353', 'hypotensive', 'patf'), ('C0687676', 'after', 'tmco'), ('C0701802', 'bike accident', 'inpo'), ('C0150312', 'present', 'qnco'), ('C0000737', 'abdominal pain', 'sosy'), ('C0039231', 'tachycardia', 'fndg'), ('C0443289', 'reveal', 'qlco'), ('C0019065', 'intraperitoneal hemorrhage', 'patf'), ('C0000726', 'abdominal', 'blor'), ('C0205447', 'one', 'qnco'), ('C0580836', 'old', 'tmco'), ('C1706180', 'male', 'qlco')], '25': [('C0019602', 'his', 'aapp, bacs, phsu'), ('C1821572', 'decreased level of consciousness', 'fndg'), ('C0018884', 'helmet', 'mnob'), ('C0087111', 'treatment', 'topp'), ('C0018670', 'head', 'blor'), ('C0205253', 'immediate', 

In [14]:
goodSty = ["Clinical Drug", "Disease or Syndrome", "Injury or Poisoning", "Pharmacologic Substance", "Sign or Symptom", "Therapeutic or Preventive Procedure"]
goodTui = [sty_d_str[gSty] for gSty in goodSty]
print (goodTui)

['T200', 'T047', 'T037', 'T121', 'T184', 'T061']


In [15]:
print("filter good original concept")
qOrigConcepts_good_d = dict()
for q, Concepts_d in qidConcepts_d.iteritems():
    for concProps in Concepts_d:
        for sty_ in concProps[2].split(','):
            if sty_d_abbr[sty_.strip()] in goodTui:
                #print(sty_, sty_d_abbr[sty_.strip()])
                if q in qOrigConcepts_good_d:
                    qOrigConcepts_good_d[q].append(concProps)
                else:
                    qOrigConcepts_good_d[q]= [concProps]
print(qOrigConcepts_good_d)

filter good original concept
{'24': [('C0278140', 'severe pain', 'sosy'), ('C0038000', 'ruptured spleen', 'inpo'), ('C0848377', 'abdominal trauma', 'inpo'), ('C0701802', 'bike accident', 'inpo'), ('C0000737', 'abdominal pain', 'sosy')], '25': [('C0019602', 'his', 'aapp, bacs, phsu'), ('C0087111', 'treatment', 'topp')], '26': [('C0020971', 'immunizations', 'topp'), ('C0199244', 'malaria prophylaxis', 'topp')], '27': [('C0009274', 'colectomies', 'topp')], '20': [('C1579887', 'car accident', 'inpo'), ('C0000924', 'accident, nos', 'inpo'), ('C0016655', 'multiple fractures', 'inpo'), ('C0234246', 'rebound abdominal pain', 'sosy')], '21': [('C0003862', 'arthralgias', 'sosy'), ('C0015672', 'fatigue', 'sosy')], '22': [('C0694551', 'abdominal pain, right lower quadrant', 'sosy'), ('C0232462', 'decreased appetite', 'sosy')], '23': [('C0239134', 'productive cough', 'sosy'), ('C0030054', 'oxygen', 'elii, phsu'), ('C0231835', 'tachypnea', 'sosy')], '28': [('C0002453', 'amenorrhea', 'dsyn'), ('C0567

In [56]:
print("extract 1st layer good related concept")
countLim = 0
qRelConcepts_good_1 = dict()
qRelConcepts_good_mutualCount_1 = dict()
for qNo, origConcs in qOrigConcepts_good_d.iteritems():
    for origConc in origConcs:
        #print(qidConcepts_d)
        if origConc[0] in relConcepts:
            relConcepts1conc = relConcepts[origConc[0]]
            #print(origConc, relConcepts1conc)
            for relCui in relConcepts1conc.keys():
                # if the concept has an acceptable name and good semantic type
                if relCui in conceptNames and bool(styConcepts[relCui]&set(goodTui)):
                    t = (relCui, list(conceptNames[relCui])[0], ','.join(styConcepts[relCui]))
                    #print(qNo)
                    #break
                    if qNo in qRelConcepts_good_1:
                        #print("----")
                        qRelConcepts_good_1[qNo].append(t)
                    else:
                        qRelConcepts_good_1[qNo] = [t]
    if qNo in qRelConcepts_good_1:
        l = qRelConcepts_good_1[qNo]
        qRelConcepts_good_mutualCount_1[qNo] = list(set([x for x in l if l.count(x) > countLim]))
        #print(qNo, list(set([(l.count(x), list(x)[1]) for x in l if l.count(x) > 1])))
#qWeightsLayerConcepts[qNo].append ([0.05, origConcs])
print(qRelConcepts_good_mutualCount_1)

extract 1st layer good related concept
{'24': [], '25': [], '26': [], '27': [], '20': [], '21': [], '22': [], '23': [], '28': [], '29': [('C0037199', 'sinusitis', 'T047'), ('C0017601', 'glaucoma', 'T047'), ('C0040822', 'tremor', 'T184'), ('C0029456', 'osteoporosis', 'T047')], '1': [], '5': [], '4': [], '7': [], '6': [], '8': [], '11': [], '10': [], '13': [], '12': [], '15': [], '17': [('C0019209', 'hepatomegaly', 'T184'), ('C0019158', 'hepatitis', 'T047')], '16': [], '18': [], '30': []}


In [57]:
print("combine different level of concepts")
qWeightsLayerConcepts = dict()
for qNo, qText in origQueries.iteritems():
    qWeightsLayerConcepts[qNo] = [[0.95, qText]]
    if qNo in qOrigConcepts_good_d:
        origConcs_good = qOrigConcepts_good_d[qNo]
        qWeightsLayerConcepts[qNo].append ([0.05, origConcs_good])
    if qNo in qRelConcepts_good_1:
        relConcepts_good_1 = qRelConcepts_good_1[qNo]
        qWeightsLayerConcepts[qNo].append ([0.01, relConcepts_good_1])

print(qWeightsLayerConcepts)

combine different level of concepts
{'24': [[0.95, u' 33-year-old male presents with severe abdominal pain one week after a bike accident in which he sustained abdominal trauma. He is hypotensive and tachycardic, and imaging reveals a ruptured spleen and intraperitoneal hemorrhage.'], [0.05, [('C0278140', 'severe pain', 'sosy'), ('C0038000', 'ruptured spleen', 'inpo'), ('C0848377', 'abdominal trauma', 'inpo'), ('C0701802', 'bike accident', 'inpo'), ('C0000737', 'abdominal pain', 'sosy')]], [0.01, [('C0020532', 'hypersplenism', 'T047'), ('C0206369', 'splenosis', 'T047'), ('C3203359', 'rupture', 'T037'), ('C0012691', 'dislocations', 'T037'), ('C0043246', 'laceration', 'T037'), ('C0035361', 'retropneumoperitoneum', 'T037'), ('C0004760', 'dysbarism', 'T037'), ('C0009938', 'contusions', 'T037'), ('C3203359', 'rupture', 'T037'), ('C0013142', 'drowning', 'T037'), ('C0016736', 'frostbite', 'T037'), ('C0016658', 'fracture', 'T037'), ('C0000924', 'accidents', 'T037'), ('C0013456', 'earache', 'T1

In [58]:
def generateIndriQueries(queryfile, qWeightsLayersConcepts, indexDir, goodTui):
    print("queryfile =", queryfile)
    regex = re.compile('[%s]' % re.escape(string.punctuation))
    with open(queryfile, "w") as f:
        f.write("<parameters>\n<index>" + indexDir + "</index>\n")
        for q, wLaysConcs in qWeightsLayersConcepts.iteritems():
            f.write("<query>\n")
            f.write("<number>" + str(q) + "</number>\n")
            f.write("<text>\n")
            f.write("\t#weight(\n")
            for i, wLayConcs in enumerate(wLaysConcs):
                #print("---> ", wLayConcs)
                wLay = wLayConcs[0]
                layConcs = wLayConcs[1]
                if i == 0:
                    layConcs = regex.sub(' ', layConcs)
                    #layConcs = str(layConcs).translate(table, string.punctuation)
                    f.write("\t\t" + str(wLay) + " #combine(" + layConcs + ")\n" )
                else:
                    concs = []
                    for concProps in layConcs:
                        concs.append(regex.sub(' ', concProps[1]))
                    if len(concs) > 0:
                        f.write("\t\t" + str(wLay) + " #weight(\n" )
                        for conc in concs:
                            #print(conc)
                            #conc = concProps[1].translate(None, string.punctuation)
                            f.write("\t\t\t" + str(1.0) + " #weight(\n" )
                            f.write("\t\t\t\t" + str(0.8) + " #combine(" + conc + ")\n")
                            f.write("\t\t\t\t" + str(0.1) + " #combine(" )
                            concStrs = conc.split()
                            concStrs1 = [concStrs[ii] + " " + concStrs[ii+1] for ii in range(0, len(concStrs)-1)]
                            if len(concStrs1) > 0:
                                for concStr in concStrs1:
                                    f.write("#od4(" + concStr.strip() + ")")
                            else:
                                f.write(conc)
                            f.write(")\n")
                            f.write("\t\t\t\t" + str(0.1) + " #uw17(" + conc + ")\n")
                            f.write("\t\t\t)\n" )
                        f.write("\t\t)\n" )
            f.write("\t)\n")
            f.write("</text>\n")
            f.write("</query>\n")
        f.write("<rule>method:two</rule>\n")
        f.write("<count>1000</count>\n")
        f.write("<threads>30</threads>\n")
        f.write("<trecFormat>true</trecFormat>\n")
        f.write("</parameters>")
"""
<goodSTYs>
Clinical Drug
Disease or Syndrome
Injury or Poisoning
Pharmacologic Substance
Sign or Symptom
Therapeutic or Preventive Procedure
</goodSTYs>
"""

'\n<goodSTYs>\nClinical Drug\nDisease or Syndrome\nInjury or Poisoning\nPharmacologic Substance\nSign or Symptom\nTherapeutic or Preventive Procedure\n</goodSTYs>\n'

In [59]:
generateIndriQueries(cds14Indriconfigs, qWeightsLayerConcepts, indexDir, goodTui)

queryfile = /home/fj9124/projects/ir/seq_kb_ir/configs/cds14/indriRunQuery.cfg


In [60]:
with open(cds14Indriconfigs, 'r') as f:
    lines = f.read()
    print(lines)

<parameters>
<index>/scratch/index/indri_5_7/pmc_mod/</index>
<query>
<number>24</number>
<text>
	#weight(
		0.95 #combine( 33 year old male presents with severe abdominal pain one week after a bike accident in which he sustained abdominal trauma  He is hypotensive and tachycardic  and imaging reveals a ruptured spleen and intraperitoneal hemorrhage )
		0.05 #weight(
			1.0 #weight(
				0.8 #combine(severe pain)
				0.1 #combine(#od4(severe pain))
				0.1 #uw17(severe pain)
			)
			1.0 #weight(
				0.8 #combine(ruptured spleen)
				0.1 #combine(#od4(ruptured spleen))
				0.1 #uw17(ruptured spleen)
			)
			1.0 #weight(
				0.8 #combine(abdominal trauma)
				0.1 #combine(#od4(abdominal trauma))
				0.1 #uw17(abdominal trauma)
			)
			1.0 #weight(
				0.8 #combine(bike accident)
				0.1 #combine(#od4(bike accident))
				0.1 #uw17(bike accident)
			)
			1.0 #weight(
				0.8 #combine(abdominal pain)
				0.1 #combine(#od4(abdominal pain))
				0.1 #uw17(abdominal pain)
			)
		)
		0.01 #weight

In [61]:
print("run query")
indriRunQueryRes = subprocess.check_output(["IndriRunQuery", cds14Indriconfigs])
print(indriRunQueryRes)

run query
24 Q0 3748403 1 -14.2918 indri
24 Q0 3825967 2 -14.3299 indri
24 Q0 2596791 3 -14.335 indri
24 Q0 3295742 4 -14.3777 indri
24 Q0 3199939 5 -14.3841 indri
24 Q0 2705345 6 -14.3907 indri
24 Q0 2673472 7 -14.4009 indri
24 Q0 3200155 8 -14.4041 indri
24 Q0 3860647 9 -14.4078 indri
24 Q0 3259121 10 -14.4322 indri
24 Q0 3423661 11 -14.4564 indri
24 Q0 3145084 12 -14.4639 indri
24 Q0 3876498 13 -14.47 indri
24 Q0 3785339 14 -14.4742 indri
24 Q0 3492969 15 -14.4772 indri
24 Q0 3336896 16 -14.4919 indri
24 Q0 3765831 17 -14.4924 indri
24 Q0 3866073 18 -14.4994 indri
24 Q0 2966580 19 -14.4995 indri
24 Q0 3016754 20 -14.5074 indri
24 Q0 3014821 21 -14.5082 indri
24 Q0 3215995 22 -14.5089 indri
24 Q0 3649317 23 -14.5103 indri
24 Q0 3635222 24 -14.5127 indri
24 Q0 3649285 25 -14.5138 indri
24 Q0 2799964 26 -14.5161 indri
24 Q0 2475515 27 -14.522 indri
24 Q0 2277435 28 -14.5244 indri
24 Q0 2815823 29 -14.5252 indri
24 Q0 3562572 30 -14.5272 indri
24 Q0 2992067 31 -14.5285 indri
24 Q0 33550

In [62]:
print("writing results on a file")
indriRunQueryResFileName = os.path.join(cds14resultsDir, "indriRunQuery.res")
print(indriRunQueryResFileName)
with open(indriRunQueryResFileName, 'w') as resFile:
    resFile.write(indriRunQueryRes)

writing results on a file
/home/fj9124/projects/ir/seq_kb_ir/results/cds14/indriRunQuery.res


In [63]:
print("calculating precision values")
cds14qrelsSampleval = os.path.join(cds14qrelsDir, "qrels-sampleval-2014.txt")
sampleEval = os.path.join(evaluatorsDir, "sample_eval.pl")
print(sampleEval)
print(cds14qrelsSampleval)
cmd = [sampleEval, cds14qrelsSampleval, indriRunQueryResFileName]
print(cmd)
samplevalRes = subprocess.check_output(cmd)
print(samplevalRes)

calculating precision values
/home/fj9124/projects/ir/evaluators/sample_eval.pl
/home/fj9124/projects/ir/seq_kb_ir/qrels/cds14/qrels-sampleval-2014.txt
['/home/fj9124/projects/ir/evaluators/sample_eval.pl', '/home/fj9124/projects/ir/seq_kb_ir/qrels/cds14/qrels-sampleval-2014.txt', '/home/fj9124/projects/ir/seq_kb_ir/results/cds14/indriRunQuery.res']
infAP		all		0.0586
infNDCG		all		0.2150
iprec@rec0.00	all		0.6649
iprec@rec0.10	all		0.1724
iprec@rec0.20	all		0.0831
iprec@rec0.30	all		0.0314
iprec@rec0.40	all		0.0294
iprec@rec0.50	all		0.0291
iprec@rec0.60	all		0.0232
iprec@rec0.70	all		0.0000
iprec@rec0.80	all		0.0000
iprec@rec0.90	all		0.0000
iprec@rec1.00	all		0.0000
iP10		all		0.3511
iP50		all		0.2404
iP100		all		0.2061
inum_rel_ret	all	      618.3661
inum_rel	all	     6705.5324
num_ret		all	     3000



In [64]:
print("writing precision values on a file") 
samplevalResFileName = os.path.join(cds14precResultsDir, "sampleEval.res") 
print(samplevalResFileName) 
with open(samplevalResFileName, 'w') as f: 
    f.write(samplevalRes)

writing precision values on a file
/home/fj9124/projects/ir/seq_kb_ir/results/cds14/sampleEval.res


In [65]:
print ("parse precision file")
with open(samplevalResFileName, 'r') as f: 
    reader = csv.reader(f, delimiter='\t', skipinitialspace=True)
    precisionVals = list(reader)
# remove empty strings:
precisionVals = [filter(None, i) for i in precisionVals]
precisionVals_d = {p[0]: float(p[2]) for p in precisionVals}
print('infNDCG =', precisionVals_d['infNDCG'])

parse precision file
infNDCG = 0.215
